#P1 turn will be taken by AI and P2 will be taken from user

In [7]:
import sys
import random
from math import inf
from functools import lru_cache

class CatchUpGame:
    def __init__(self, n):
        """Initialize game with n numbers (1 to n)"""
        # Safety: Limit n to prevent performance issues
        self.n = min(n, 1000)  # Maximum allowed value
        if n > 1000:
            print("Note: n capped at 1000 for performance")

        # Game state tracking
        self.remaining_numbers = set(range(1, self.n + 1))  # Numbers left to pick
        self.p1_sum = 0  # Player 1's total score
        self.p2_sum = 0  # Player 2's total score
        self.current_player = 1  # 1 for P1, 2 for P2
        self.p1_last_turn_sum = 0  # Sum P1 got in their last complete turn
        self.p2_last_turn_sum = 0  # Sum P2 got in their last complete turn
        self.current_turn_sum = 0  # Sum being accumulated in current turn
        self.turn_count = 0  # Total turns completed

    def get_available_numbers(self):
        """Returns sorted list of remaining numbers"""
        return sorted(self.remaining_numbers)

    def make_move(self, number):
        """
        Process a number selection:
        1. Validates the number is available
        2. Updates turn sums
        3. Checks if turn should end
        """
        if number not in self.remaining_numbers:
            raise ValueError("Number not available")

        # Update game state
        self.remaining_numbers.remove(number)
        self.current_turn_sum += number

        # Determine target sum to reach (opponent's last turn sum)
        target = self.p2_last_turn_sum if self.current_player == 1 else self.p1_last_turn_sum

        # Check if turn should end (target reached or no numbers left)
        if self.current_turn_sum >= target or not self.remaining_numbers:
            # Update player's total sum
            if self.current_player == 1:
                self.p1_sum += self.current_turn_sum
                self.p1_last_turn_sum = self.current_turn_sum
            else:
                self.p2_sum += self.current_turn_sum
                self.p2_last_turn_sum = self.current_turn_sum

            # Switch players and reset turn
            self.current_player = 3 - self.current_player  # Switches between 1 and 2
            self.current_turn_sum = 0
            self.turn_count += 1

    def is_game_over(self):
        """Check if all numbers have been picked"""
        return not self.remaining_numbers

    def evaluate(self):
        """Heuristic for Minimax: P1 wants to maximize (p1_sum - p2_sum)"""
        return self.p1_sum - self.p2_sum

    @lru_cache(maxsize=None)  # Cache results for performance
    def minimax(self, depth, alpha, beta, maximizing_player):
        """
        Minimax algorithm with Alpha-Beta pruning:
        - depth: How many moves ahead to look
        - alpha: Best already explored for maximizer
        - beta: Best already explored for minimizer
        - maximizing_player: True if current player is P1 (maximizer)
        """
        # Base case: game over or depth limit reached
        if self.is_game_over() or depth == 0:
            return self.evaluate()

        available_numbers = self.get_available_numbers()
        if not available_numbers:
            return 0  # No moves left

        if maximizing_player:  # P1's turn (maximize score)
            max_eval = -inf
            for num in available_numbers:
                # Simulate move
                game_copy = self.copy()
                try:
                    game_copy.make_move(num)
                    # Recursive evaluation with pruning
                    eval = game_copy.minimax(depth - 1, alpha, beta, False)
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)  # Update alpha
                    if beta <= alpha:
                        break  # Beta pruning - stop exploring this branch
                except ValueError:
                    continue  # Skip invalid moves
            return max_eval
        else:  # P2's turn (minimize score)
            min_eval = inf
            for num in available_numbers:
                # Simulate move
                game_copy = self.copy()
                try:
                    game_copy.make_move(num)
                    # Recursive evaluation with pruning
                    eval = game_copy.minimax(depth - 1, alpha, beta, True)
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)  # Update beta
                    if beta <= alpha:
                        break  # Alpha pruning - stop exploring this branch
                except ValueError:
                    continue  # Skip invalid moves
            return min_eval

    def copy(self):
        """Create a deep copy of the current game state"""
        new_game = CatchUpGame(self.n)
        # Copy all relevant attributes
        new_game.remaining_numbers = self.remaining_numbers.copy()
        new_game.p1_sum = self.p1_sum
        new_game.p2_sum = self.p2_sum
        new_game.current_player = self.current_player
        new_game.p1_last_turn_sum = self.p1_last_turn_sum
        new_game.p2_last_turn_sum = self.p2_last_turn_sum
        new_game.current_turn_sum = self.current_turn_sum
        new_game.turn_count = self.turn_count
        return new_game

def get_user_input(prompt, min_val=None, max_val=None):
    """
    Get validated user input:
    - Ensures input is an integer
    - Validates against min/max bounds
    - Prevents extremely large n values
    """
    while True:
        try:
            value = int(input(prompt))
            # Validate against limits
            if min_val is not None and value < min_val:
                print(f"Value must be ≥ {min_val}")
                continue
            if max_val is not None and value > max_val:
                print(f"Maximum allowed value is {max_val}")
                continue
            return value
        except ValueError:
            print("Please enter a valid number")

def play_game():
    """Main game loop"""
    print("Enter the value of n (1-1000): ", end="")
    n = get_user_input("", 1, 1000)  # Force n between 1-1000
    game = CatchUpGame(n)

    print(f"\n=== Catch-Up Game (n={n}) ===")
    print("Rules:")
    print("- P1 picks first (1 number only)")
    print("- Subsequent turns: pick until your turn sum ≥ opponent's last turn sum")

    # Game loop
    while not game.is_game_over():
        current_player = "P1" if game.current_player == 1 else "P2"
        opponent = "P2" if game.current_player == 1 else "P1"
        target = game.p2_last_turn_sum if game.current_player == 1 else game.p1_last_turn_sum

        # Display game state
        print(f"\nAvailable numbers: {game.get_available_numbers()}")
        print(f"P1 total: {game.p1_sum} | P2 total: {game.p2_sum}")

        # First move special case
        if game.turn_count == 0 and game.current_player == 1:
            num = get_user_input("P1, pick your FIRST number (1 only): ", 1, game.n)
            game.make_move(num)
            continue

        # Normal turn
        print(f"{current_player}'s turn. Need sum ≥ {target}")
        print(f"Current turn sum: {game.current_turn_sum}/{target}")

        # AI move (P1 as AI in this example)
        if current_player == "P1":

            available_numbers = game.get_available_numbers()
            best_num = None
            best_eval = -inf if game.current_player == 1 else inf

            # Evaluate all possible moves
            for num in available_numbers:
                game_copy = game.copy()
                try:
                    game_copy.make_move(num)
                    eval = game_copy.minimax(3, -inf, inf, game.current_player == 2)
                    # Update best move
                    if (game.current_player == 1 and eval > best_eval) or \
                       (game.current_player == 2 and eval < best_eval):
                        best_eval = eval
                        best_num = num
                except ValueError:
                    continue

            num = best_num if best_num else random.choice(available_numbers)
            print(f"P1 chooses: {num}")
        else:
            # Human move
            num = get_user_input("P2 chooses: ", 1, game.n)

        # Process the move
        try:
            game.make_move(num)
        except ValueError:
            print("Invalid move! Try again")
            continue

    # Game over
    print("\n=== Game Over ===")
    print(f"Final scores - P1: {game.p1_sum} | P2: {game.p2_sum}")
    if game.p1_sum > game.p2_sum:
        print("P1 wins!")
    elif game.p2_sum > game.p1_sum:
        print("P2 wins!")
    else:
        print("It's a tie!")

if __name__ == "__main__":
    play_game()

Enter the value of n (1-1000): 10

=== Catch-Up Game (n=10) ===
Rules:
- P1 picks first (1 number only)
- Subsequent turns: pick until your turn sum ≥ opponent's last turn sum

Available numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
P1 total: 0 | P2 total: 0
P1, pick your FIRST number (1 only): 3

Available numbers: [1, 2, 4, 5, 6, 7, 8, 9, 10]
P1 total: 3 | P2 total: 0
P2's turn. Need sum ≥ 3
Current turn sum: 0/3
P2 chooses: 4

Available numbers: [1, 2, 5, 6, 7, 8, 9, 10]
P1 total: 3 | P2 total: 4
P1's turn. Need sum ≥ 4
Current turn sum: 0/4
P1 chooses: 1

Available numbers: [2, 5, 6, 7, 8, 9, 10]
P1 total: 3 | P2 total: 4
P1's turn. Need sum ≥ 4
Current turn sum: 1/4
P1 chooses: 6

Available numbers: [2, 5, 7, 8, 9, 10]
P1 total: 10 | P2 total: 4
P2's turn. Need sum ≥ 7
Current turn sum: 0/7
P2 chooses: 8

Available numbers: [2, 5, 7, 9, 10]
P1 total: 10 | P2 total: 12
P1's turn. Need sum ≥ 8
Current turn sum: 0/8
P1 chooses: 5

Available numbers: [2, 7, 9, 10]
P1 total: 10 | P2 total: 

#Both P1 and P2  are taking input from user

In [13]:
import sys
import random
from math import inf
from functools import lru_cache

class CatchUpGame:
    def __init__(self, n):
        """Initialize game with n numbers (1 to n)"""
        # Safety: Limit n to prevent performance issues
        self.n = min(n, 1000)  # Maximum allowed value
        if n > 1000:
            print("Note: n capped at 1000 for performance")

        # Game state tracking
        self.remaining_numbers = set(range(1, self.n + 1))  # Numbers left to pick
        self.p1_sum = 0  # Player 1's total score
        self.p2_sum = 0  # Player 2's total score
        self.current_player = 1  # 1 for P1, 2 for P2
        self.p1_last_turn_sum = 0  # Sum P1 got in their last complete turn
        self.p2_last_turn_sum = 0  # Sum P2 got in their last complete turn
        self.current_turn_sum = 0  # Sum being accumulated in current turn
        self.turn_count = 0  # Total turns completed

    def get_available_numbers(self):
        """Returns sorted list of remaining numbers"""
        return sorted(self.remaining_numbers)

    def make_move(self, number):
        """
        Process a number selection:
        1. Validates the number is available
        2. Updates turn sums
        3. Checks if turn should end
        """
        if number not in self.remaining_numbers:
            raise ValueError("Number not available")

        # Update game state
        self.remaining_numbers.remove(number)
        self.current_turn_sum += number

        # Determine target sum to reach (opponent's last turn sum)
        target = self.p2_last_turn_sum if self.current_player == 1 else self.p1_last_turn_sum

        # Check if turn should end (target reached or no numbers left)
        if self.current_turn_sum >= target or not self.remaining_numbers:
            # Update player's total sum
            if self.current_player == 1:
                self.p1_sum += self.current_turn_sum
                self.p1_last_turn_sum = self.current_turn_sum
            else:
                self.p2_sum += self.current_turn_sum
                self.p2_last_turn_sum = self.current_turn_sum

            # Switch players and reset turn
            self.current_player = 3 - self.current_player  # Switches between 1 and 2
            self.current_turn_sum = 0
            self.turn_count += 1

    def is_game_over(self):
        """Check if all numbers have been picked"""
        return not self.remaining_numbers

    def evaluate(self):
        """Heuristic for Minimax: P1 wants to maximize (p1_sum - p2_sum)"""
        return self.p1_sum - self.p2_sum

    @lru_cache(maxsize=None)  # Cache results for performance
    def minimax(self, depth, alpha, beta, maximizing_player):
        """
        Minimax algorithm with Alpha-Beta pruning:
        - depth: How many moves ahead to look
        - alpha: Best already explored for maximizer
        - beta: Best already explored for minimizer
        - maximizing_player: True if current player is P1 (maximizer)
        """
        # Base case: game over or depth limit reached
        if self.is_game_over() or depth == 0:
            return self.evaluate()

        available_numbers = self.get_available_numbers()
        if not available_numbers:
            return 0  # No moves left

        if maximizing_player:  # P1's turn (maximize score)
            max_eval = -inf
            for num in available_numbers:
                # Simulate move
                game_copy = self.copy()
                try:
                    game_copy.make_move(num)
                    # Recursive evaluation with pruning
                    eval = game_copy.minimax(depth - 1, alpha, beta, False)
                    max_eval = max(max_eval, eval)
                    alpha = max(alpha, eval)  # Update alpha
                    if beta <= alpha:
                        break  # Beta pruning - stop exploring this branch
                except ValueError:
                    continue  # Skip invalid moves
            return max_eval
        else:  # P2's turn (minimize score)
            min_eval = inf
            for num in available_numbers:
                # Simulate move
                game_copy = self.copy()
                try:
                    game_copy.make_move(num)
                    # Recursive evaluation with pruning
                    eval = game_copy.minimax(depth - 1, alpha, beta, True)
                    min_eval = min(min_eval, eval)
                    beta = min(beta, eval)  # Update beta
                    if beta <= alpha:
                        break  # Alpha pruning - stop exploring this branch
                except ValueError:
                    continue  # Skip invalid moves
            return min_eval

    def copy(self):
        """Create a deep copy of the current game state"""
        new_game = CatchUpGame(self.n)
        # Copy all relevant attributes
        new_game.remaining_numbers = self.remaining_numbers.copy()
        new_game.p1_sum = self.p1_sum
        new_game.p2_sum = self.p2_sum
        new_game.current_player = self.current_player
        new_game.p1_last_turn_sum = self.p1_last_turn_sum
        new_game.p2_last_turn_sum = self.p2_last_turn_sum
        new_game.current_turn_sum = self.current_turn_sum
        new_game.turn_count = self.turn_count
        return new_game

def get_user_input(prompt, min_val=None, max_val=None):
    """
    Get validated user input:
    - Ensures input is an integer
    - Validates against min/max bounds
    - Prevents extremely large n values
    """
    while True:
        try:
            value = int(input(prompt))
            # Validate against limits
            if min_val is not None and value < min_val:
                print(f"Value must be ≥ {min_val}")
                continue
            if max_val is not None and value > max_val:
                print(f"Maximum allowed value is {max_val}")
                continue
            return value
        except ValueError:
            print("Please enter a valid number")

def play_game():
    """Main game loop"""
    print("Enter the value of n (1-1000): ", end="")
    n = get_user_input("", 1, 1000)  # Force n between 1-1000
    game = CatchUpGame(n)

    print(f"\n=== Catch-Up Game (n={n}) ===")
    print("Rules:")
    print("- P1 picks first (1 number only)")
    print("- Subsequent turns: pick until your turn sum ≥ opponent's last turn sum")

    # Game loop
    while not game.is_game_over():
        current_player = "P1" if game.current_player == 1 else "P2"
        opponent = "P2" if game.current_player == 1 else "P1"
        target = game.p2_last_turn_sum if game.current_player == 1 else game.p1_last_turn_sum

        # Display game state
        print(f"\nAvailable numbers: {game.get_available_numbers()}")
        print(f"P1 total: {game.p1_sum} | P2 total: {game.p2_sum}")

        # First move special case
        if game.turn_count == 0 and game.current_player == 1:
            num = get_user_input("P1, pick your FIRST number (1 only): ", 1, game.n)
            game.make_move(num)
            continue

        # Normal turn
        print(f"{current_player}'s turn. Need sum ≥ {target}")
        print(f"Current turn sum: {game.current_turn_sum}/{target}")

        # Get human move for both players
        num = get_user_input(f"{current_player} chooses: ", 1, game.n)


        # Process the move
        try:
            game.make_move(num)
        except ValueError:
            print("Invalid move! Try again")
            continue

    # Game over
    print("\n=== Game Over ===")
    print(f"Final scores - P1: {game.p1_sum} | P2: {game.p2_sum}")
    if game.p1_sum > game.p2_sum:
        print("P1 wins!")
    elif game.p2_sum > game.p1_sum:
        print("P2 wins!")
    else:
        print("It's a tie!")
if __name__ == "__main__":
    play_game()


Enter the value of n (1-1000): 15

=== Catch-Up Game (n=15) ===
Rules:
- P1 picks first (1 number only)
- Subsequent turns: pick until your turn sum ≥ opponent's last turn sum

Available numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
P1 total: 0 | P2 total: 0
P1, pick your FIRST number (1 only): gh
Please enter a valid number
P1, pick your FIRST number (1 only): 0
Value must be ≥ 1
P1, pick your FIRST number (1 only): 7

Available numbers: [1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
P1 total: 7 | P2 total: 0
P2's turn. Need sum ≥ 7
Current turn sum: 0/7
P2 chooses: 2

Available numbers: [1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
P1 total: 7 | P2 total: 0
P2's turn. Need sum ≥ 7
Current turn sum: 2/7
P2 chooses: 3

Available numbers: [1, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
P1 total: 7 | P2 total: 0
P2's turn. Need sum ≥ 7
Current turn sum: 5/7
P2 chooses: 4

Available numbers: [1, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15]
P1 total: 7 | P2 total: 9
P1's turn. Need sum 